<a href="https://colab.research.google.com/github/DavidZyy/dive-into-ml/blob/main/gradient_desent_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 梯度下降算法

## 梯度下降法（gradient descent）

梯度下降法可以用来求解线性方程：
$$
Ax - b = 0
$$
但是要求系数矩阵$A$是实对称正定矩阵。即有$A^T = A$，$x^
TAx \geq 0$。
我们可以重新把问题表述为求一个二次型最小值的问题，
目标函数为：
$$
F(x) = x^T A x - 2 x^T b,
$$

$F(x)$是一个二次型，有梯度：
$$
\nabla F(x) = 2 (Ax - b)
$$
使得梯度梯度为0的$x$，即为二次型的极值点。同时也是上述线性方程的解。

于是我们可以得到如下的迭代算法：我们任意选择一个$x$作为初始值，求出$x$点的梯度$r$，然后选择步长$\gamma$，得到新的$x = x-\gamma r$，如此迭代多次，$x$最终能够收敛到极值点。

那么如何选择步长$\gamma$呢？ 由我们已经得到了梯度的方向$(Ax-b)$，省略掉系数。令$r = b - Ax $，为梯度的负方向。使得$F$在当前梯度方向上最小的步长即为最优步长。相当于最小化以下函数：
$$
F(x_k + \gamma r_k)
$$
我们可以对$\gamma$求导，使得导数等于0，最终求得：
$$
\gamma = \frac{\mathbf{r}^\top \mathbf{r}}{\mathbf{r}^\top \mathbf{A} \mathbf{r}}
$$
证明见附录。

于是可以得到以下算法：


\begin{aligned}&{\text{repeat in the loop:}}\\&\qquad \mathbf {r} :=\mathbf {b} -\mathbf {Ax} \\&\qquad \gamma :={\mathbf {r} ^{\mathsf {T}}\mathbf {r} }/{\mathbf {r} ^{\mathsf {T}}\mathbf {Ar} }\\&\qquad \mathbf {x} :=\mathbf {x} +\gamma \mathbf {r} \\&\qquad {\hbox{if }}\mathbf {r} ^{\mathsf {T}}\mathbf {r} {\text{ is sufficiently small, then exit loop}}\\&{\text{end repeat loop}}\\&{\text{return }}\mathbf {x} {\text{ as the result}}\end{aligned}

先写一个程序用以生成$A$和$b$：

In [1]:
import numpy as np

# Set a seed for reproducibility
np.random.seed(42)

# Define the size of the matrix and vectors
n = 3  # You can change the size as needed

# Step 1: Generate a random matrix M
M = np.random.randint(0, 10, size=(n, n))

# Step 2: Form A as M^T M to ensure it's symmetric positive definite
A = np.dot(M.T, M)

# Step 3: Generate a random solution vector x (true solution)
x_true = np.random.randint(0, 10, size = n)

# Step 4: Compute the corresponding b
b = np.dot(A, x_true)

print("Matrix A:")
print(A)
print("\nVector b:")
print(b)

# Step 5: Solve for x using a linear solver
x_computed = np.linalg.solve(A, b)

print("\nComputed solution x:")
print(x_computed)

# Verify that the computed solution is close to the true solution
print("\nTrue solution x (for verification):")
print(x_true)


Matrix A:
[[ 56  54  92]
 [ 54  81 117]
 [ 92 117 179]]

Vector b:
[1030 1278 1972]

Computed solution x:
[4. 3. 7.]

True solution x (for verification):
[4 3 7]


In [2]:
def iterative_solver(A, b, tol=1e-6, max_iter=10000):
    # Initialize variables
    x = np.zeros_like(b)
    r = b - np.dot(A, x)
    iteration = 0

    while np.dot(r, r) > tol and iteration < max_iter:
        # Compute gamma
        gamma = np.dot(r, r) / np.dot(r, np.dot(A, r))
        # Update x
        x = x + gamma * r
        # Update r
        r = b - np.dot(A, x)
        # Increment iteration counter
        iteration += 1
        # print(f"Iteration {iteration}: x = {x}, r = {r}")

    return x, iteration

# Solve the system
solution, iters = iterative_solver(A, b)


# Print the result and the number of iterations
print(f"Solution: {solution}")
print(f"Iterations: {iters}")

Solution: [4.00521064 3.00728266 6.99255962]
Iterations: 2497


为了避免同一次迭代中两次乘$A$，可以对算法进行优化，原理见附录：


\begin{aligned}&\mathbf {r} :=\mathbf {b} -\mathbf {Ax} \\&{\text{repeat in the loop:}}\\&\qquad \gamma :={\mathbf {r} ^{\mathsf {T}}\mathbf {r} }/{\mathbf {r} ^{\mathsf {T}}\mathbf {Ar} }\\&\qquad \mathbf {x} :=\mathbf {x} +\gamma \mathbf {r} \\&\qquad {\hbox{if }}\mathbf {r} ^{\mathsf {T}}\mathbf {r} {\text{ is sufficiently small, then exit loop}}\\&\qquad \mathbf {r} :=\mathbf {r} -\gamma \mathbf {Ar} \\&{\text{end repeat loop}}\\&{\text{return }}\mathbf {x} {\text{ as the result}}\end{aligned}


# 附录

## 步长的证明

求
$$
F(x_k + \gamma r_k)
$$
的最小值，参数是$\gamma$，其中$r_k = b - Ax_k$。
展开
$$
(x + \gamma r)^T A (x + \gamma r) - 2(x+ \gamma r ) ^T b
$$

为了简洁，去掉下标$k$，使用$\lambda$替代$\gamma$，有：
$$
F(x + \lambda r) = (x + \lambda r)^\top A (x + \lambda r) - 2 (x + \lambda r)^\top b
$$

继续展开：
$$
 x^\top A x + \lambda r^\top A x + \lambda x^\top A r + \lambda^2 r^\top A r - 2 x^\top b - 2 \lambda r^\top b
 $$

对$\lambda$求导：
$$
r^\top A x + x^\top A r + 2\lambda r^\top A r - 2 r^\top b
$$

替换$b = Ax + r$。
$$
r^\top A x + x^\top A r + 2\lambda r^\top A r - 2 r^\top (Ax + r)
$$
其中可以证明
$$
r^\top A x = x^\top A r
$$

最终化简得到：
$$
\gamma = \lambda = \frac{\mathbf{r}^\top \mathbf{r}}{\mathbf{r}^\top \mathbf{A} \mathbf{r}}
$$

## 算法优化

The statement "we note that $\mathbf{x} := \mathbf{x} + \gamma \mathbf{r}$ implies $\mathbf{r} := \mathbf{r} - \gamma \mathbf{A} \mathbf{r}$" can be explained through the iterative update steps in the algorithm.

Let's go through the derivation step-by-step.

1. **Initial Residual Calculation:**
   $$
   \mathbf{r} := \mathbf{b} - \mathbf{A}\mathbf{x}
   $$

2. **Update Step for $\mathbf{x}$:**
   $$
   \mathbf{x} := \mathbf{x} + \gamma \mathbf{r}
   $$

3. **New Residual Calculation:**
   The residual $\mathbf{r}$ needs to be updated after $\mathbf{x}$ is updated. The new residual can be derived as follows:

   $$
   \mathbf{r}_{\text{new}} = \mathbf{b} - \mathbf{A}\mathbf{x}_{\text{new}}
   $$

4. **Substitute the Updated $\mathbf{x}$:**
   From the update step:
   $$
   \mathbf{x}_{\text{new}} = \mathbf{x} + \gamma \mathbf{r}
   $$

   So,
   $$
   \mathbf{r}_{\text{new}} = \mathbf{b} - \mathbf{A}(\mathbf{x} + \gamma \mathbf{r})
   $$

5. **Expand and Simplify:**
   $$
   \mathbf{r}_{\text{new}} = \mathbf{b} - \mathbf{A}\mathbf{x} - \gamma \mathbf{A}\mathbf{r}
   $$

6. **Recognize the Original Residual:**
   Recall that the original residual $\mathbf{r} = \mathbf{b} - \mathbf{A}\mathbf{x}$, so:
   $$
   \mathbf{r}_{\text{new}} = \mathbf{r} - \gamma \mathbf{A}\mathbf{r}
   $$

Therefore, after updating $\mathbf{x}$ with $\mathbf{x} := \mathbf{x} + \gamma \mathbf{r}$, the residual $\mathbf{r}$ can be updated using $\mathbf{r} := \mathbf{r} - \gamma \mathbf{A} \mathbf{r}$.

This relationship shows how the residual changes as $\mathbf{x}$ is iteratively updated in the algorithm.


# 参考资料
[1] [Gradient descent](https://en.wikipedia.org/wiki/Gradient_descent)